<a href="https://colab.research.google.com/github/ground-Jiang/DNA-sequences-translation/blob/main/DNA_sequences_into_protein_sequences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.3 MB/s eta 0:00:00


In [2]:
#@title **input your DNA sequences in the right box**

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Input and output file paths
input_file = 'Pseudomonas_aeruginosa_PAO1_ORF.ffn' #@param {type:"string"}
# Replace with the path to your .ffn file
output_file = '/content/output_file.fasta'  # Replace with the path to save the .fasta file

# List to hold translated protein sequences
protein_records = []

# Read the DNA sequences from the .ffn file
for dna_record in SeqIO.parse(input_file, "fasta"):
    # Translate the DNA sequence into a protein sequence
    protein_seq = dna_record.seq.translate()

    # Create a SeqRecord for the protein sequence
    protein_record = SeqRecord(protein_seq, id=dna_record.id, description="translated protein")

    # Add the protein record to the list
    protein_records.append(protein_record)

# Write the protein sequences to a .fasta file
SeqIO.write(protein_records, output_file, "fasta")

print(f"Protein sequences saved to {output_file}")


/usr/local/lib/python3.10/dist-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Protein sequences saved to /content/output_file.fasta
